# 01. 순환 신경망(Recurrent Neural Network, RNN)

* RNN(Recurrent Neural Network)은 시퀀스(Sequence) 모델
* 입력과 출력을 시퀀스 단위로 처리하는 모델

* 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향으로만 향하는 신경망들을 피드 포워드 신경망(Feed Forward Neural Network)이라고 한다. 
*  RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고있다.
* RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 셀(cell)이라고 한다. 이 셀은 이전의 값을 기억하려고 하는 일종의 **메모리 역할**을 수행하므로 이를 메모리 셀 또는 RNN 셀이라고 표현합니다.

은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있습니다. 앞으로는 현재 시점을 변수 t로 표현하겠습니다. 이는 현재 시점 t에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미합니다. 메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값을 은닉 상태(hidden state)라고 합니다. 다시 말해 t 시점의 메모리 셀은 t-1 시점의 메모리 셀이 보낸 은닉 상태값을 t 시점의 은닉 상태 계산을 위한 입력값으로 사용합니다.

 <img src="https://wikidocs.net/images/page/22886/rnn_image3_ver2.PNG" width="400" height="150">

RNN 셀의 각 시점 별 입, 출력의 단위는 사용자가 정의하기 나름이지만 가장 보편적인 단위는 '단어 벡터'입니다.

### one-to-many 일대다
하나의 이미지 입력에 대해서 사진의 제목을 출력하는 이미지 캡셔닝(Image Captioning) 작업에 사용

 ### many to one 다대일
* 입력 문서가 긍정적인지 부정적인지를 판별하는 감성 분류(sentiment classification), 또는 메일이 정상 메일인지 스팸 메일인지 판별하는 스팸 메일 분류(spam detection)에 사용  
 <img src="https://wikidocs.net/images/page/22886/rnn_image3.5.PNG" width="300" height="200">

### many-to-many 다대다
* 입력 문장으로 부터 대답 문장을 출력하는 챗봇과 입력 문장으로부터 번역된 문장을 출력하는 번역기, 개체명 인식이나 품사 태깅과 같은 작업이 속합니다. 위 그림은 개체명 인식을 수행할 때의 RNN 아키텍처를 보여줍니다.
 <img src="https://wikidocs.net/images/page/22886/rnn_image3.7.PNG" width="200" height="150">

 <img src="https://wikidocs.net/images/page/22886/rnn_image4_ver2.PNG" width="200" height="200">  
 
* 현재 시점 t에서의 은닉 상태값을 $h_{t}$라고 정의. 은닉층의 메모리 셀은 $h_{t}$를 계산하기 위해서 총 두 개의 가중치를 갖게 된다.
* 하나는 입력층에서 입력값을 위한 가중치 $W_{x}$이고, 하나는 이전 시점 t-1의 은닉 상태값인 $h_{t-1}$을 위한 가중치 $W_{h}$다.

<img src="https://wikidocs.net/images/page/22886/rnn_images4-5.PNG" width="400" height="100">  